In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
)

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)



INFO 08-26 22:25:21 [__init__.py:244] Automatically detected platform cuda.
INFO 08-26 22:25:35 [config.py:841] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 08-26 22:25:35 [config.py:1472] Using max model len 131072
WARNING 08-26 22:25:35 [arg_utils.py:1577] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
WARNING 08-26 22:25:35 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 08-26 22:25:35 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qw

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 08-26 22:25:38 [default_loader.py:272] Loading weights took 1.19 seconds
INFO 08-26 22:25:39 [model_runner.py:1255] Model loading took 3.3461 GiB and 1.355255 seconds
INFO 08-26 22:25:43 [worker.py:295] Memory profiling takes 3.98 seconds
INFO 08-26 22:25:43 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 08-26 22:25:43 [worker.py:295] model weights take 3.35GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 8.07GiB; the rest of the memory reserved for KV Cache is 31.22GiB.
INFO 08-26 22:25:43 [executor_base.py:115] # cuda blocks: 73063, # CPU blocks: 9362
INFO 08-26 22:25:43 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 8.92x
INFO 08-26 22:25:46 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 7.83 seconds


In [2]:
# Define the suffix for newline tokens in the tokenizer
target_suffix = "ĊĊ"  # "\n\n" is tokenized as "ĊĊ"

# Get complete tokenizer vocabulary
vocab = tokenizer.get_vocab()

# Find all tokens and their IDs that end with the target suffix
# These are the newline tokens we'll apply steering to
matching_tokens_ids = [
    token_id
    for token, token_id in vocab.items()
    if isinstance(token, str) and token.endswith(target_suffix)
]

# Configure steering vector request for SEAL control
sv_request = SteerVectorRequest(
    # Name and ID for the steering vector
    steer_vector_name="complex_control",
    steer_vector_id=4,
    
    # Configure the three steering vectors (execution, reflection, transition)
    vector_configs=[
        # Execution vector (positive scale to promote execution-like text)
        VectorConfig(
            path="execution_avg_vector.gguf",
            scale=0.5,                            # Positive scale promotes this behavior
            target_layers=[20],                   # Apply at layer 20
            generate_trigger_tokens=matching_tokens_ids,  # Apply to newline tokens
            algorithm="direct",                   # Direct application
            normalize=False                       # Do not normalize vectors
        ),
        
        # Reflection vector (negative scale to suppress reflection)
        VectorConfig(
            path="reflection_avg_vector.gguf",
            scale=-0.5,                           # Negative scale suppresses this behavior
            target_layers=[20],
            generate_trigger_tokens=matching_tokens_ids,
            algorithm="direct",
            normalize=False
        ),
        
        # Transition vector (negative scale to suppress transitions)
        VectorConfig(
            path="transition_avg_vector.gguf",
            scale=-0.5,                           # Negative scale suppresses this behavior
            target_layers=[20],
            generate_trigger_tokens=matching_tokens_ids,
            algorithm="direct", 
            normalize=False
        ),
    ],
    
    # Additional parameters
    debug=False,                        # Don't output debug info
    conflict_resolution="sequential"    # Apply vectors in sequence
)

# MATH500

In [2]:
import json
file_path = "/home/xhl/eval/my_eval/data/math500/test.jsonl"

problems = []
answers = []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        problems.append(item["problem"])
        answers.append(item["answer"])

# 看看前两个
print("Problems:", problems[:2])
print("Answers:", answers[:2])


examples = ["Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + prompt + "\nAssistant: <think>" for prompt in problems]


Problems: ['Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$', 'Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$']
Answers: ['\\left( 3, \\frac{\\pi}{2} \\right)', 'p - q']


In [ ]:
# Generate response with SEAL steering
example_answers = llm.generate(
    examples, 
    SamplingParams(
        temperature=0,
        max_tokens=8192,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

In [17]:
from math_verify import parse, verify, LatexExtractionConfig, ExprExtractionConfig
outputs = [output.outputs[0].text for output in example_answers]
extraction_target = (ExprExtractionConfig(), LatexExtractionConfig())
results = []
for i, llm_output in enumerate(outputs):
    gold = parse(f"${answers[i]}$", extraction_config=extraction_target)
    answer = parse(llm_output, extraction_config=extraction_target)
    result = verify(gold, answer)
    results.append(result)
accuracy = sum(results) / len(results)
print(accuracy)

0.784


In [18]:
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
)
length = 0
for i in range(len(outputs)):
    length += len(tokenizer.tokenize(outputs[i], add_special_tokens=True))
print("Length: ", length/len(outputs))

Length:  3074.668


# GSM8k

In [3]:
import json
file_path = "/home/xhl/eval/my_eval/data/gsm8k/test.jsonl"

problems = []
answers = []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        problems.append(item["question"])
        answers.append(item["answer"])

# 看看前两个
print("Problems:", problems[:2])
print("Answers:", answers[:2])


examples = ["Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + prompt + "\nAssistant: <think>" for prompt in problems]


Problems: ["Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?']
Answers: ['Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18', 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3']


In [ ]:
example_answers = llm.generate(
    examples, 
    SamplingParams(
        temperature=0,
        max_tokens=8192,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

In [5]:
from math_verify import parse, verify, LatexExtractionConfig, ExprExtractionConfig
outputs = [output.outputs[0].text for output in example_answers]
extraction_target = (ExprExtractionConfig(), LatexExtractionConfig())
results = []
for i, llm_output in enumerate(outputs):
    gold = parse(f"${answers[i]}$", extraction_config=extraction_target)
    answer = parse(llm_output, extraction_config=extraction_target)
    result = verify(gold, answer)
    results.append(result)
accuracy = sum(results) / len(results)
print(accuracy)

0.8233510235026535


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
)
length = 0
for i in range(len(outputs)):
    length += len(tokenizer.tokenize(outputs[i], add_special_tokens=True))
print("Length: ", length/len(outputs))

Length:  1460.1266110689917
